In [1]:
from pprint import pprint

In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

2025/12/21 15:23:28 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/21 15:23:28 INFO mlflow.store.db.utils: Updating database tables
2025/12/21 15:23:28 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/21 15:23:28 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/21 15:23:28 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/21 15:23:28 INFO alembic.runtime.migration: Will assume non-transactional DDL.


In [13]:
for i, item in enumerate(client.search_experiments()):
    print(f"Item {i}:")
    pprint(item)
    print("-" * 40)

Item 0:
<Experiment: artifact_location='/home/ludo/Public/Projets_IA/MLOps_ZoomCamp/02-experiment-tracking/mlruns/1', creation_time=1765564082300, experiment_id='1', last_update_time=1765564082300, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>
----------------------------------------
Item 1:
<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1765549614040, experiment_id='0', last_update_time=1765549614040, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>
----------------------------------------


In [15]:
from mlflow.entities import ViewType


runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 4650c9a8af3e49d3aad80452a95ff93a, rmse: 6.3086
run id: 87e881e16a654d60bd6b348a9c16a0e5, rmse: 6.3086
run id: 09ef01a9d8ce437187df671e23f9f03e, rmse: 6.3086
run id: b6be8332e3a742068da167c561905704, rmse: 6.3086
run id: 76641f799c214d7098b89d0140d7b181, rmse: 6.3111


In [16]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "b6be8332e3a742068da167c561905704"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-model")

2025/12/21 15:36:44 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/21 15:36:45 INFO mlflow.store.db.utils: Updating database tables
2025/12/21 15:36:45 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/21 15:36:45 INFO alembic.runtime.migration: Will assume non-transactional DDL.
Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...


MlflowException: Unable to find a logged_model with artifact_path model under run b6be8332e3a742068da167c561905704

In [17]:
model_name = "nyc-taxi-model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None


/tmp/ipykernel_1357824/29955185.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


See also: https://github.com/ziritrion/mlopszoomcamp/blob/main/notes/2_experiment.md